## Dataset

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

In [ ]:
# Função para carregar e preprocessar os dados
def load_and_preprocess_data(csv_file, time_steps, scaler):
    # Carregar o dataset com cabeçalho
    data = pd.read_csv(csv_file)
    
    # Converter a coluna de timestamp para o formato datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S')
    data.set_index('timestamp', inplace=True)

    # A variável de entrada (X) será a coluna 'activity'
    X = data[['activity']].values  # Usando 'activity' como feature
    # A variável alvo (y) será a coluna 'activity'
    y = data['activity'].values  # 'activity' como alvo
    
    # Normalizar as variáveis de entrada
    X = scaler.fit_transform(X)
    y = scaler.transform(y.reshape(-1, 1))
    # Criar sequências para o modelo LSTM
    X_seq, y_seq = create_sequences(X, y, time_steps)
    return X_seq, y_seq

In [ ]:
def load_and_preprocess_data_test(csv_file, time_steps, scaler):
    # Carregar o dataset com cabeçalho
    
    data = pd.read_csv(csv_file)
    concat_file = csv_file[:-8] + '_nov.csv'

    concat_data = pd.read_csv(concat_file)
    
    data = pd.concat([concat_data.tail(time_steps), data])
    # Converter a coluna de timestamp para o formato datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S')
    data.set_index('timestamp', inplace=True)
    # A variável de entrada (X) será a coluna 'activity'
    X = data[['activity']].values  # Usando 'activity' como feature
    # A variável alvo (y) será a coluna 'activity'
    y = data['activity'].values  # 'activity' como alvo
    
    # Normalizar as variáveis de entrada
    X = scaler.fit_transform(X)
    y = scaler.transform(y.reshape(-1, 1))
    # Criar sequências para o modelo LSTM
    X_seq, y_seq = create_sequences(X, y, time_steps)
    # print(data.head(31))
    # print(scaler.inverse_transform(y_seq[0].reshape(-1, 1)))
    return X_seq, y_seq

In [ ]:
# Função para criar sequências temporais
def create_sequences(X, y, time_steps):
    sequences = []
    targets = []
    for i in range(len(X) - time_steps):
        # print(X[i:i + time_steps], y[i + time_steps])
        sequences.append(X[i:i + time_steps])  # Sequência de time_steps
        targets.append(y[i + time_steps])  # Alvo correspondente ao final da sequência
    return np.array(sequences), np.array(targets)

In [ ]:
def calculate_metrics(y_true, y_pred):
    ae = (y_pred - y_true)
    mse = np.mean((y_true - y_pred) ** 2)
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(mse)

In [ ]:
def gerar_dados_para_river(X_train, y_train):
    for i in range(len(X_train)):
        # Convertendo a amostra para um dicionário { 'feature_0': valor, 'feature_1': valor, ... }
        x_dict = {f'feature_{j}': X_train[i][j][0] for j in range(X_train.shape[1])}
        
        # Extraindo o valor escalar de y
        y_value = y_train[i][0]
        
        yield x_dict, y_value  # Gerador de pares (x, y)

## Modelos

In [ ]:
from river import datasets
from river import evaluate
from river import forest
from river import metrics
from river import preprocessing

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import math
from river import forest, metrics, tree, ensemble, linear_model, preprocessing
from hyperopt import fmin, tpe, hp, Trials

In [ ]:
from pympler import asizeof
import time

In [ ]:
# Criar a pasta 'resultados' se não existir
os.makedirs("resultados", exist_ok=True)

### AMFRegressor

In [ ]:
AMFRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'    
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = forest.AMFRegressor(seed=42)

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)
 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = [] 
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/AMF_{i}.csv', index=False)

    AMFRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
})

### ARFRegressor

In [ ]:
ARFRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20  # Tamanho dos lotes temporais
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = (
                forest.ARFRegressor(seed=42)
            )

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/ARF_{i}.csv', index=False)

    ARFRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### OXTRegressor

In [ ]:
OXTRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'    
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = forest.OXTRegressor(seed=42)

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB    

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/OTX_{i}.csv', index=False)

    OXTRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### BaggingRegressor

In [ ]:
from river import datasets
from river import ensemble
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing

In [ ]:
BaggingRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = preprocessing.StandardScaler()
    model |= ensemble.BaggingRegressor(
        model=linear_model.LinearRegression(intercept_lr=0.1),
        seed=42
    )

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB
    
    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/BaR_{i}.csv', index=False)

    BaggingRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### EWARegressor

In [ ]:
from river import datasets
from river import ensemble
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing
from river import stream

In [ ]:
EWARegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv' 
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = (
        linear_model.LinearRegression(
            optimizer=optim.AdaGrad(),
            intercept_lr=.1
        )
    )

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

       # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/EWA_{i}.csv', index=False)

    EWARegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### SRPRegressor

In [ ]:
from river import ensemble
from river import evaluate
from river import metrics
from river.datasets import synth
from river import tree

In [ ]:
SRPRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    base_model = tree.HoeffdingTreeRegressor()
    model = ensemble.SRPRegressor(
        model=base_model,
        training_method="patches",
        seed=42
    )

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/SRP_{i}.csv', index=False)

    SRPRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### HardSamplingRegressor

In [ ]:
from river import evaluate
from river import imblearn
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing

In [ ]:
HardSamplingRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = (
    imblearn.HardSamplingRegressor(
        regressor=linear_model.LinearRegression(),
        p=.2,
        size=30,
        seed=42,
    )
)

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/HSR_{i}.csv', index=False)

    HardSamplingRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### HoeffdingAdaptiveTreeRegressor

In [ ]:
HoeffdingAdaptiveTreeregressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv' 
    chunk_size = 20 
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = tree.HoeffdingAdaptiveTreeRegressor( seed=42 )

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/HAT_{i}.csv', index=False)

    HoeffdingAdaptiveTreeregressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### HoeffdingTreeRegressor

In [ ]:
HoeffdingTreeRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = tree.HoeffdingTreeRegressor()

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/HTR_{i}.csv', index=False)

    HoeffdingTreeRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### SGTRegressor

In [ ]:
SGTRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv' 
    chunk_size = 20 
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = tree.SGTRegressor()

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/SGT_{i}.csv', index=False)

    SGTRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### BayesianLinearRegression

In [ ]:
BayesianLinearRegression = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = linear_model.BayesianLinearRegression()

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1)))


        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

        # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/BLR_{i}.csv', index=False)

    BayesianLinearRegression.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### LinearRegression

In [ ]:
LinearRegression = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = linear_model.LinearRegression()

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/LiR_{i}.csv', index=False)

    LinearRegression.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### LogisticRegression

In [ ]:
LogisticRegression = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = linear_model.LogisticRegression()
    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB
    
    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/LoR_{i}.csv', index=False)

    LogisticRegression.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### PARegressor

In [ ]:
PARegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'  
    chunk_size = 20  
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = linear_model.PARegressor()

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/PAR_{i}.csv', index=False)

    PARegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### Perceptron

In [ ]:
from river import linear_model as lm

In [ ]:
Perceptron = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'    
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = lm.Perceptron()

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(np.array(y_pred).reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB
    
    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/Per_{i}.csv', index=False)

    Perceptron.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### SoftmaxRegression

In [ ]:
SoftmaxRegression = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv' 
    chunk_size = 20   
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = linear_model.SoftmaxRegression()

    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  

    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/SoR_{i}.csv', index=False)

    SoftmaxRegression.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### KNNRegressor

In [ ]:
from river import neighbors

In [ ]:
KNNRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv'   
    chunk_size = 20 
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = neighbors.KNNRegressor()
    
    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

 
    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  
    
    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/KNN_{i}.csv', index=False)

    KNNRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

### SKL2RiverRegressor

In [ ]:
from river import compat
from sklearn import linear_model

In [ ]:
SKL2RiverRegressor = []

In [ ]:
for i in range(0,5):
    train_csv = 'data/clstr_0_nov.csv'
    test_csv = f'data/clstr_{i}_dec.csv' 
    chunk_size = 20   
    scaler = StandardScaler()
    test_scaler = StandardScaler()
    X_train, y_train = load_and_preprocess_data(train_csv, chunk_size, scaler)
    X_test, y_test = load_and_preprocess_data_test(test_csv, chunk_size, test_scaler)

    model = compat.convert_sklearn_to_river(linear_model.SGDRegressor())
    
    mse_metric = metrics.MSE()
    mae_metric = metrics.MAE()
    rmse_metric = metrics.RMSE()

    # Criando o gerador de dados
    dados_river = gerar_dados_para_river(X_train, y_train)

    for xi, yi in dados_river:            
                # Treinar o modelo com o dado atual
                model.learn_one(xi, yi)
  
    data = gerar_dados_para_river(X_test, y_test)

    # Listas para armazenar previsões e valores reais
    predictions = []
    target = []
    resultados = []

    tempo_inferencias = []
    tempo_aprendiazagem = []

    for xi, yi in data:
        t0 = time.perf_counter()
        y_pred = model.predict_one(xi)
        t1 = time.perf_counter()
        tempo_inferencias.append(t1 - t0)
        predictions.append(y_pred if y_pred is not None else np.nan)
        target.append(yi)
                    
        # Escrever a previsão no arquivo
        yi_unnorm = float(test_scaler.inverse_transform(yi.reshape(-1, 1)))
        y_pred_unnorm = float(test_scaler.inverse_transform(y_pred.reshape(-1, 1)))

        # Atualizar a métrica com a predição
        if y_pred is not None:
            mse_metric.update(yi_unnorm, y_pred_unnorm)
            mae_metric.update(yi_unnorm, y_pred_unnorm)
            rmse_metric.update(yi_unnorm, y_pred_unnorm)
            
        # Guardar os resultados na lista
        resultados.append({
            "Real": yi_unnorm,
            "Previsto": y_pred_unnorm,
            "Erro Absoluto": abs(y_pred_unnorm - yi_unnorm),
            "MAE Atual": mae_metric.get(),
            "MSE Atual": mse_metric.get(),
            "RMSE Atual": rmse_metric.get()
        })

        # Treinar o modelo com o dado atual
        ti = time.perf_counter()
        model.learn_one(xi, yi)
        tf = time.perf_counter()
        tempo_aprendiazagem.append(tf - ti) 

    # >>> Medir o uso de memória
    tamanho_memoria = asizeof.asizeof(model) / 1024  # em KB

    # Converter para DataFrame e exibir
    df_resultados = pd.DataFrame(resultados)

    # Salvar o DataFrame em um arquivo CSV
    df_resultados_csv = pd.DataFrame({
    "Real_Requests": df_resultados["Real"].round().astype(int),
    "Predicted_Requests": df_resultados["Previsto"].round().astype(int)
    })

    df_resultados_csv.to_csv(f'resultados/SKL_{i}.csv', index=False)

    SKL2RiverRegressor.append({
    "Cluster": i,
    "MAE": mae_metric.get(),
    "RMSE": rmse_metric.get(),
    "Tempo Médio de Inferência (ms)": round(np.mean(tempo_inferencias) * 1000, 4),  # >>> em milissegundos
    "Tempo Aprendizagem (ms)": round(np.mean(tempo_aprendiazagem) * 1000, 4),
    "Tamanho do Modelo (KB)": round(tamanho_memoria, 2)
    })

## Resultados

In [ ]:
import pandas as pd

# Lista com os nomes das variáveis de resultados dos modelos
modelos_resultados = [
    ("AMFRegressor", AMFRegressor),
    ("ARFRegressor", ARFRegressor),
    ("OXTRegressor", OXTRegressor),
    ("BaggingRegressor", BaggingRegressor),
    ("EWARegressor", EWARegressor),
    ("SRPRegressor", SRPRegressor),
    ("HardSamplingRegressor", HardSamplingRegressor),
    ("HoeffdingAdaptiveTreeregressor", HoeffdingAdaptiveTreeregressor),
    ("HoeffdingTreeRegressor", HoeffdingTreeRegressor),
    ("SGTRegressor", SGTRegressor),
    ("BayesianLinearRegression", BayesianLinearRegression),
    ("LinearRegression", LinearRegression),
    ("LogisticRegression", LogisticRegression),
    ("PARegressor", PARegressor),
    ("Perceptron", Perceptron),
    ("SoftmaxRegression", SoftmaxRegression),
    ("KNNRegressor", KNNRegressor),
    ("SKL2RiverRegressor", SKL2RiverRegressor)
]

# Tabelas para armazenar os dados
tabela_metricas = []
tabela_tempos = []
tabela_memoria = []

# Iterar sobre os resultados
for nome_modelo, resultados in modelos_resultados:
    mae = [r.get('MAE', 0.0) for r in resultados]
    rmse = [r.get('RMSE', 0.0) for r in resultados]
    inf = [r.get('Tempo Médio de Inferência (ms)', 0.0) for r in resultados]
    learn = [r.get('Tempo Aprendizagem (ms)', 0.0) for r in resultados]
    size = [r.get('Tamanho do Modelo (KB)', 0.0) for r in resultados]

    # Tabela de métricas
    tabela_metricas.append([nome_modelo, 'MAE'] + mae)
    tabela_metricas.append(['', 'RMSE'] + rmse)

    # Tabela de tempos
    tabela_tempos.append([nome_modelo, 'Inferência (ms)'] + inf)
    tabela_tempos.append(['', 'Aprendizagem (ms)'] + learn)

    # Tabela de memória
    tabela_memoria.append([nome_modelo, 'Tamanho do Modelo (KB)'] + size)

# Converter para DataFrames
colunas = ['Modelo', 'Métrica', 'Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']
df_metricas = pd.DataFrame(tabela_metricas, columns=colunas)
df_tempos = pd.DataFrame(tabela_tempos, columns=colunas)
df_memoria = pd.DataFrame(tabela_memoria, columns=colunas)

# Adicionar coluna de média
for df in [df_metricas, df_tempos, df_memoria]:
    df['Média'] = df.iloc[:, 2:7].mean(axis=1)


In [ ]:
with pd.ExcelWriter("resultados_completos_modelos.xlsx") as writer:
    df_metricas.to_excel(writer, sheet_name="Métricas", index=False)
    df_tempos.to_excel(writer, sheet_name="Tempos", index=False)
    df_memoria.to_excel(writer, sheet_name="Tamanho do Modelo", index=False)
